In [13]:
import qiskit
import numpy as np
from qiskit import QuantumCircuit , execute , Aer, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram , plot_bloch_multivector
from math import sqrt, pi

q = QuantumRegister(3)
c = ClassicalRegister(2)
ckt = QuantumCircuit(q, c)
backend = Aer.get_backend('qasm_simulator')
shots = 10

N = 11
theta = np.linspace(0, pi, N)
phi = np.linspace(0, pi/2, N)
# maximum entanglement
r = pi/2

strategies = [(i, j) for i in theta for j in phi]
payoff_matrix = np.array([[3,3], [0,5], [5,0], [1,1]])

strategy_pair = {}
payoff_pair = {}
for theta1, phi1 in strategies:
    U1 = np.array([[np.exp(phi1*1j)*np.cos(theta1/2),np.sin(theta1/2)],[-np.sin(theta1/2),np.exp(-phi1*1j)*np.cos(theta1/2)]])
    payoff_max = 0
    for theta2, phi2 in strategies:
        U2 = np.array([[np.exp(phi2*1j)*np.cos(theta2/2),np.sin(theta2/2)],[-np.sin(theta2/2),np.exp(-phi2*1j)*np.cos(theta2/2)]])
        # clear the quantum circuit
        ckt.data.clear()
        ckt.reset(q)
        # end
        ckt.x(0)  #C = 1
        ckt.x(1)  #C = 1
        ckt.rx(-pi/2, 0)
        ckt.rx(-pi/2, 1)
        ckt.cx(q[0], q[2])
        ckt.cx(q[1], q[2])
        ckt.rz(r, q[2])
        ckt.cx(q[1], q[2])
        ckt.cx(q[0], q[2])
        ckt.rx(pi/2, 1)
        ckt.rx(pi/2, 0)
        ckt.squ(U1, q[0])
        ckt.squ(U2, q[1])
        ckt.rx(-pi/2, 0)
        ckt.rx(-pi/2, 1)
        ckt.cx(q[0], q[2])
        ckt.cx(q[1], q[2])
        ckt.rz(-r, q[2])
        ckt.cx(q[1], q[2])
        ckt.cx(q[0], q[2])
        ckt.rx(pi/2, 1)
        ckt.rx(pi/2, 0)
        ckt.measure([0,1], [0,1])
        results = execute(ckt, backend=backend, shots=shots).result()
        answer = results.get_counts()
        payoff = 0
        for i in answer:
            if (i == "11"):
                payoff = payoff + payoff_matrix[0]*answer[i]/shots
            if (i == "10"):
                payoff = payoff + payoff_matrix[2]*answer[i]/shots
            if (i == "01"):
                payoff = payoff + payoff_matrix[1]*answer[i]/shots
            if (i == "00"):
                payoff = payoff + payoff_matrix[3]*answer[i]/shots
        if payoff[1] > payoff_max:
            strategy_pair[(theta1, phi1)] = (theta2, phi2)
            payoff_pair[(theta1, phi1)] = payoff
            payoff_max = payoff[1]

# NE
# if player 1 choose (theta1, phi1), then player 2 will choose (theta2, phi2)
# if player 2 choose (theta2, phi2), then player 1 will choose (theta1, phi1)
## by symmetric, the second condition can be re-stated as follows
## if player 1 choose (theta2, phi2), then player 2 will choose (theta1, phi1)
NE = [(k, v) for (k, v) in strategy_pair.items() if strategy_pair[v] == k]
for cnt, ne in enumerate(NE):
    print(f'******NE #{cnt+1}******')
    print(f'Player 1: theta = {ne[0][0]}, phi = {ne[0][1]}')
    print(f'Player 2: theta = {ne[1][0]}, phi = {ne[1][1]}')
    print('Payoff: ', payoff_pair[ne[0]])

#print(NE)

******NE #1******
Player 1: theta = 0.0, phi = 1.5707963267948966
Player 2: theta = 0.0, phi = 1.5707963267948966
Payoff:  [3. 3.]


In [7]:
theta

array([0.        , 0.31415927, 0.62831853, 0.9424778 , 1.25663706,
       1.57079633, 1.88495559, 2.19911486, 2.51327412, 2.82743339,
       3.14159265])

In [8]:
phi

array([0.        , 0.15707963, 0.31415927, 0.4712389 , 0.62831853,
       0.78539816, 0.9424778 , 1.09955743, 1.25663706, 1.41371669,
       1.57079633])